<a href="https://colab.research.google.com/github/ZorkNo/AIGame/blob/master/ImageTextCompCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#CLIP
!pip install clip-by-openai
#AWS 
!pip install boto3

## more sacure way to save passwords
!pip install colab-env --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.2 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 12.8 MB 17.3 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.7.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.

In [2]:
!pip install opensearch-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204 kB 4.2 MB/s 
     |████████████████████████████████| 127 kB 38.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11


In [3]:
import torch
import clip

import cv2

from PIL import Image

import math
import numpy as np
import pandas as pd

import boto3

import colab_env
import os

import datetime
from opensearchpy import OpenSearch, helpers
import click

Mounted at /content/gdrive


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "ViT-B/32"
print(clip.available_models())
print('Loading clip model...')
model, preprocess = clip.load(model_name, device=device)
print('done.')

['RN50', 'ViT-B/32']
Loading clip model...


100%|██████████████████████| 353976522/353976522 [00:02<00:00, 167636092.03it/s]


done.


In [5]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-69c4c91c-4fc8-29b2-8cab-8e60a5fe35a1)


In [6]:
def makedir(path):
  if os.path.exists(path) == False :
    os.mkdir(path)

def save_image(parent_dir,dir,image_file_name,cvimage):
  #make directories
  makedir('/'+parent_dir+'/')
  makedir('/'+parent_dir+'/'+dir+'/')
  #saving file
  return cv2.imwrite('/'+parent_dir+'/'+dir+'/'+image_file_name+'',cvimage)

s3 = boto3.resource('s3')
bucket = s3.Bucket('vframes')

def upload(parent_dir,dir,image_file_name,bucketname,cvimage):
  save_image(parent_dir,dir,image_file_name,cvimage)

  path = os.path.join('/',parent_dir,dir,image_file_name)
  s3path = os.path.join(parent_dir,dir,image_file_name)
  print ((path))
  #print (os.path.exists(path))
  s3.meta.client.upload_file(path,bucketname,s3path)

In [7]:
# Function that computes the feature vectors for a batch of images
def compute_clip_features(photos):
    
    # Preprocess all photos
    photos_preprocessed = torch.stack([preprocess(photo) for photo in photos]).to(device)

    with torch.no_grad():
        # Encode the photos batch to compute the feature vectors and normalize them
        photos_features = model.encode_image(photos_preprocessed)
        photos_features /= photos_features.norm(dim=-1, keepdim=True)

    # Transfer the feature vectors back to the CPU and convert to numpy
    return photos_features.cpu().numpy()

In [8]:

# Function that convert from cv2 to pillow image
def convert_from_cv2_to_image(img: np.ndarray) -> Image:
    # return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    return Image.fromarray(img)

def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    # return cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)
    return np.asarray(img)

In [9]:
class FeatureTimespan:
  def __init__(self, title, prdNo,url,sourceUrl,imageUrl,frameStart,frameEnd,featureFrame,features):
    self.title = title
    self.prdNo = prdNo
    self.url = url
    self.sourceUrl  = sourceUrl
    self.imageUrl = imageUrl
    self.frameStart  = frameStart
    self.frameEnd  = frameEnd
    self.featureFrame  = featureFrame
    self.features = features

In [19]:
usr = os.getenv("AWS_OPENSEARCH_USR")
psw = os.getenv("AWS_OPENSEARCH_PSW")

def load_frames_in_index(featureTimespans, es_url, index_name):
    es = OpenSearch(es_url, http_auth=(usr, psw), port=80, request_timeout=400,
                       max_retries=10, retry_on_timeout=True, use_ssl = True)
    now = datetime.datetime.utcnow()
    length = len(featureTimespans)
    actions = ({
        "_index": index_name,
        "title": features.title,
        "prdNo": features.prdNo,
        "url": features.url,
        "sourceUrl": features.sourceUrl,
        "imageUrl": features.imageUrl,
        "frameStart": features.frameStart,
        "frameEnd": features.frameEnd,
        "featureFrame": features.featureFrame,
        "features": features.features,
        "@timestamp": now
    } for features in featureTimespans)

    loading = helpers.streaming_bulk(
        client=es,
        actions=actions,
        max_retries=5
    )
    print(f'Loading {length} photos...', flush=True)
    with click.progressbar(loading, length=length) as task:
        for success, info in task:
            if not success:
                print('A document failed:', info)

    print('Loaded photos')

In [35]:
# import dependencies
from google.colab.patches import cv2_imshow
import cv2
import time
from scipy import spatial

# Define the batch size so that it fits on your GPU. You can also do the processing on the CPU, but it will be slower.
batch_size = 512
# How many frames will be encode (1 / X frame_amount = 1 is all frames
frame_amount = 4

#
administrativeTitle= 'Spise med Price S13:E2 Spise med Price - Soul Food [00652105020]'
productionNumber= '00652105020'
siteUrl= 'https://www.dr.dk/drtv/episode/spise-med-price_-soul-food_265992'
streamUrl = 'https://drod22m.akamaized.net/all/clear/none/a8/6135f36faf5a611d6c7849a8/00652105020/stream_ts/master_manifest.m3u8'
streamUrl2 ='https://drod22m.akamaized.net/all/clear/none/a8/6135f36faf5a611d6c7849a8/00652105020/stream_ts/Spise-med-Price---Soul_13520bccc26942fda22024a003ae2041_3500.m3u8'

In [ ]:
cap = cv2.VideoCapture(streamUrl2)
batchCount = 0
frame_count = 0
image_count = 0
keyframe_count = 0
images = []
frame = []
listfeatureTimespans =[]
pre_image_feature = np.empty
pre_image_feature_set = False
#pre_image = Null
pre_image_frame=1
last_frame=False

while cap.isOpened():    
    frame_count = frame_count + 1 
    ret, image = cap.read()

    if not ret:
      print('not ret') 
      last_frame =True
      #break

    #Only read every (frame_amount) frames
    if frame_count%frame_amount  == 0 or last_frame :
      #print ('f'+ str(frame_count)
      if(last_frame==False): 
        image_count = image_count + 1
        images.append(convert_from_cv2_to_image(image))
        frame.append(frame_count)

      # encode when reaches (batch_size) 
      if image_count%batch_size == 0 or (last_frame and len(images)>0):

        print('Encode')
        images_features = compute_clip_features(images)
        print('Encoded')
        j=0
        
        for imageF in images_features:

          if pre_image_feature_set==False:
            pre_image_frame=1
            pre_image_cv = image[0]
            pre_image_feature = imageF
            pre_image_feature_set=True

          result = 1 - spatial.distance.cosine(pre_image_feature, imageF)
          if result <0.85:
            objectname = str(pre_image_frame).zfill(6)+'.png'
            
            ##uploads
            upload('frames',productionNumber,objectname,'vframes',pre_image_cv)

            #writes to opensearch
            featureTimespan = FeatureTimespan("","","","","",0,0,0,np.zeros(512))
            featureTimespan.title = administrativeTitle
            featureTimespan.prdNo = productionNumber
            featureTimespan.url = siteUrl
            featureTimespan.sourceUrl = streamUrl
            featureTimespan.imageUrl ='https://vframes.s3.eu-north-1.amazonaws.com/frames/'+productionNumber+'/'+objectname
            featureTimespan.frameStart =pre_image_frame
            featureTimespan.frameEnd = frame[j]-1
            featureTimespan.featureFrame = pre_image_frame
            featureTimespan.features = pre_image_feature
            listfeatureTimespans.append(featureTimespan)
            
            pre_image_frame=frame[j]
            pre_image_cv = convert_from_image_to_cv2(images[j])
            pre_image_feature = imageF
                    
          j=j+1
        load_frames_in_index(listfeatureTimespans,"https://search-video-ovjjz53fbtmpdokttzlfxmkrpu.eu-west-2.es.amazonaws.com","image_search")
        listfeatureTimespans= []
        images = []
        frame= []


    #cv2.waitKey(1) & 0xff

cv2.destroyAllWindows()
cap.release()


Encode
Encoded
/frames/00652105020/000001.png
/frames/00652105020/000200.png
/frames/00652105020/000208.png
/frames/00652105020/000220.png
/frames/00652105020/000228.png
/frames/00652105020/000244.png
/frames/00652105020/000252.png
/frames/00652105020/000256.png
/frames/00652105020/000320.png
/frames/00652105020/000332.png
/frames/00652105020/000340.png
/frames/00652105020/000356.png
/frames/00652105020/000364.png
/frames/00652105020/000428.png
/frames/00652105020/000432.png
/frames/00652105020/000436.png
/frames/00652105020/000448.png
/frames/00652105020/000464.png
/frames/00652105020/000512.png
/frames/00652105020/000520.png
/frames/00652105020/000528.png
/frames/00652105020/000532.png
/frames/00652105020/000596.png
/frames/00652105020/000612.png
/frames/00652105020/000616.png
/frames/00652105020/000680.png
/frames/00652105020/000688.png
/frames/00652105020/000696.png
/frames/00652105020/000700.png
/frames/00652105020/000772.png
/frames/00652105020/000784.png
/frames/00652105020/0007

In [31]:
        print('Encode')
        images_features = compute_clip_features(images)
        print('Encoded')
        j=0
        
        for imageF in images_features:

          if pre_image_feature_set==False:
            pre_image_frame=1
            pre_image_cv = image[0]
            pre_image_feature = imageF
            pre_image_feature_set=True

          result = 1 - spatial.distance.cosine(pre_image_feature, imageF)
          if result <0.85:
            objectname = str(pre_image_frame).zfill(6)+'.png'
            
            ##uploads
            upload('frames',productionNumber,objectname,'vframes',pre_image_cv)

            #writes to opensearch
            featureTimespan = FeatureTimespan("","","","","",0,0,0,np.zeros(512))
            featureTimespan.title = administrativeTitle
            featureTimespan.prdNo = productionNumber
            featureTimespan.url = siteUrl
            featureTimespan.sourceUrl = streamUrl
            featureTimespan.imageUrl ='https://vframes.s3.eu-north-1.amazonaws.com/frames/'+productionNumber+'/'+objectname
            featureTimespan.frameStart =pre_image_frame
            featureTimespan.frameEnd = frame[j]-1
            featureTimespan.featureFrame = pre_image_frame
            featureTimespan.features = pre_image_feature
            listfeatureTimespans.append(featureTimespan)
            
            pre_image_frame=frame[j]
            pre_image_cv = convert_from_image_to_cv2(images[j])
            pre_image_feature = imageF
                    
          j=j+1
        load_frames_in_index(listfeatureTimespans,"https://search-video-ovjjz53fbtmpdokttzlfxmkrpu.eu-west-2.es.amazonaws.com","image_search")
        listfeatureTimespans= []
        images = []
        frame= []

Encode
Encoded
/frames/00652105010/042948.png
/frames/00652105010/043016.png
/frames/00652105010/043080.png
/frames/00652105010/043128.png
/frames/00652105010/043168.png
/frames/00652105010/043232.png
/frames/00652105010/043252.png
/frames/00652105010/043292.png
/frames/00652105010/043296.png
/frames/00652105010/043316.png
/frames/00652105010/043344.png
/frames/00652105010/043448.png
/frames/00652105010/043468.png
/frames/00652105010/043504.png
/frames/00652105010/043568.png
Loading 15 photos...
Loaded photos
